In [ ]:
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду
https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad 
    средствами библиотеки Pandas сформируйте поле выводов по каждому анализирумомому параметру. 
    Например, по показателю pH получен результат 8.4 единицы pH при нормативе от 6 до 9 единиц pH. 
    Т.о. по данному показателю результат анализа в норме. Для решения задачи необходимо программно "прочитать и понять" 
    значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. 
    Например, 6 >= pH >= 9. В итоговом DataFrame столбец "Показатель" сделайте индексным.

Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" 
https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).

In [2]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing

In [10]:


headers={
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
}
def get_content(url):
    with requests.Session() as req:
        req.headers.update(headers)
        r = req.get(url).content
    return r

url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = get_content(url)
df=pd.read_csv(io.StringIO(s.decode('UTF8')))


### Если не работает загрузка on-line
#df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [59]:
headers={
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
}
def get_content(url):
    with requests.Session() as req:
        req.headers.update(headers)
        r = req.get(url).content
    return r

url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = get_content(url)
df=pd.read_csv(io.StringIO(s.decode('UTF8')))

df.iat[0, 2] ="9,9"#для теста

stb_count= df.shape[1]  #количество столбцов
str_count = df.shape[0] #количество строк


df.insert(4, "Вывод", "")

temp_str = 0

#в цикле по всем строкам
while temp_str < str_count:
    

    #if df.Норматив[temp_str] == "не более 2-3":
    
    s = str(df.Норматив[temp_str])
    number = s.find("не более ") # номер вхождения подстроки
    #Если строка содержит "не более" 
    if number != -1:    
        
        number_tire = s.find("-")
        if number_tire  > number: # удаляем пробел
            s = s.replace(s[number+len("не более"):number_tire+1] , '')
                          
        #верхняя граница нормы
        float_value_normativ = float(s[number+len("не более"): len(s)].replace(" ", "").replace(",", "."))
        
        value = str(df.iat[temp_str, 2])
        
        
        #проверка, является ли строка числом 
        
        try:
            float_value = float(value)
        
        except: 
            df.iat[temp_str, 4] = "НЕ НОРМА"  
            float_value= None
            
        if float_value == None:
            df.iat[temp_str, 4] = "НЕ НОРМА"    
        elif float_value>float_value_normativ :
            df.iat[temp_str, 4] = "НЕ НОРМА"
        else:
            df.iat[temp_str, 4] ="Норма"        
    
    #Если строка содержит "в пределах"     
    elif s.find("в пределах") != -1:
        number = s.find("в пределах ") # номер вхождения подстроки
        
        number_tire = s.find("-")
        if number_tire  > number: # удаляем пробел
            s_low = s[number+len("в пределах"):number_tire]
            s = s.replace(s[number+len("в пределах"):number_tire+1] , '')
                          
        #верхняя граница нормы
        float_value_normativ = float(s[number+len("в пределах"): len(s)].replace(" ", "").replace(",", "."))
        
        
        float_value_normativ_low= float(s_low.replace(" ", "").replace(",", "."))
        
        value = str(df.iat[temp_str, 2])
        
        
        #проверка, является ли строка числом 
        
        try:
            float_value = float(value)
        
        except: 
            df.iat[temp_str, 4] = "НЕ НОРМА"  
            float_value= None
            
        if float_value == None:
            df.iat[temp_str, 4] = "НЕ НОРМА"    
        elif float_value>float_value_normativ :
            df.iat[temp_str, 4] = "НЕ НОРМА"
        elif float_value<float_value_normativ_low :
            df.iat[temp_str, 4] = "НЕ НОРМА"    
        else:
            df.iat[temp_str, 4] ="Норма"     
        
 

    temp_str+=1
 
    
display(df)    

,Показатель,Единица измерений,Результат анализа,Норматив,Вывод
0,pH,единицы pH,"9,9",в пределах 6-9,НЕ НОРМА
1,Запах,баллы,1,не более 2-3,Норма
2,Цветность,градусы,б/цвета,не более 30,НЕ НОРМА
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,Норма
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",Норма
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",Норма
6,Нитраты (по NO3),мг/дм3,24,не более 45,Норма
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5",Норма
8,Хлориды (Cl),мг/дм3,200,не более 350,Норма
9,Сульфаты (SO4),мг/дм3,189.5,не более 500,Норма
